
# Canal plan 2D : comparaison des méthodes numériques


In [ ]:
from trustutils import run
from trustutils.jupyter import plot
import numpy as np
from functions import *
from math import *
import matplotlib.pyplot as plt

run.introduction("M. El Moatamid","22/05/2023")

run.TRUST_parameters()

### Description

On simule un écoulement monophasique turbulent en RANS dans un canal plan avec les différentes configurations:
- VDF, VEF et PolyMAC (schémas numériques)
- k-epsilon, k-omega et k-tau (modèles de turbulence)
    
    La simulation est faite sur un canal de longueur L=100m et hauteur 2H=2m 
    
                                          symétrie
                  ______________________________________________________________
                  |                                                             |
      Inlet:      |             Bloc maillage n°3 : hauteur=0.6m                | Outlet:
        velocity->|-------------------------------------------------------------|-> pressure
        k       ->|             Bloc maillage n°2 : hauteur=0.2m                |-> k
        epsilon ->|-------------------------------------------------------------|-> epsilon
                  |             Bloc maillage n°1 : hauteur=0.2m                |
                  |_____________________________________________________________|
                                           Paroi
    
    
Il est possible de lancer le calcul sur plusieurs maillages : 

L'utilisateur choisi le nombre de noeuds selon y dans le bloc n°1 : liste du nombre de noeuds par maillage Ny

Comme les rectangles sont coupés en 4 triangles pour VEF, il y aura 2 fois plus de mailles que la valeur entrée
    
Attention : le rapport d'aspect AR={AR} est défini dans function.py. 

Pour VEF, si Ny=2 le rapport d'aspect est diviser par 2

Il est possible de déraffiner le maillage dans les bloc n°2 et 3 avec des facteurs multiplictaifs des tailles de mailles : taux1 et taux2

## Préparation des calculs
A remplir par l'utilisateur

In [ ]:
colors = [
    "red",
    "blue",
    "green",
    "yellow",
    "orange",
    "purple",
    "pink",
    "brown",
    "black",
    "white",
    "gray",
    "cyan",
    "magenta",
    "lime",
    "maroon",
    "navy",
    "olive",
    "teal",
    "aqua",
    "silver",
    "gold",
    "beige",
    "coral",
    "ivory",
    "khaki",
    "lavender",
    "salmon",
    "tan",
    "turquoise",
    "violet",
    "indigo",
    "chartreuse",
    "fuchsia",
    "azure",
    "bisque",
    "crimson",
    "orchid",
    "plum",
    "sienna",
    "tomato",
    "wheat",
]

markers = [
    "-",  # solid line style
    "--",  # dashed line style
    "-.",  # dash-dot line style
    ":",  # dotted line style
]

# Configurations
vef, vdf, plm = "VEFPreP1b", "VDF", "PolyMAC_P0"
keps, komg, ktau = "k-epsilon", "k-omega", "k-tau"
pbt, pbh, pbm = "turb", "hydr", "multi"

configs = [
    (vef, keps, pbt),
    (vdf, keps, pbt),
    # (vdf, ktau, pbh),
    (vdf, ktau, pbm),
    # (vdf, komg, pbh),
    (vdf, komg, pbm),
    (plm, ktau, pbh),
    (plm, ktau, pbm),
    (plm, komg, pbh),
    (plm, komg, pbm),
]

# Parallelisation du calcul
number_of_partitions = 8

# Fluide
mu = 0.02
rho = 1000.0

# Conditions limites
inlet_velocity = 1
outlet_pressure = 1e5
inlet_k = 0.01
inlet_epsilon = 0.005

# Maillage

# Attention il y aura 2 fois plus de cellules selon y que la valeur définie ici (trianguler_h avec vef coupe les maille en 4)
Ny = [2, 3]  # nombre de noeuds dans le 1er bloc de maillage VEF
taux1 = [1, 1, 1]  # facteur multiplicatif : taille des cellules dans le bloc n°2
taux2 = [1, 1, 1]  # facteur multiplicatif : taille des cellules dans le bloc n°3

# # Boucle temporelle
tmax = 300  # [s]

# # Post traitement

# plot profiles
x_prof = 95  # position selon x pour extraire différents profils (vitesse, k, ...)
y_min_prof = sonde_firstpoint(0.2, Ny)
y_max_prof = 1 - sonde_firstpoint(0.2, Ny)
nb_points_prof = sonde_nbpoints(0.2, Ny)

### Premier calculs sur les paramètres entrés
On calcule sur la base des paramètres de simulation choisis par l'utilisateur :
- Le nombre de Reynolds
- Une estimation du coef de frottement qui permet de prédire (bonne approximation) tau_w, utau et y⁺

L'utilisateur peut donc savoir à quels résultats s'attendre et vérifier que les valeurs de y+ permettent d'utiliser une loi de paroi log (loi standard utilisée avec VEF)
    
On vérifie également que le maillage peut être construit

In [ ]:
H = 1  # hauteur canal [m]
Re_H = Reynolds(inlet_velocity, rho, H, mu)  # calcul du reynolds
cf_pred = cf_prediction(kappa, B, Re_H, cf_guess)  # estimation du coefficient de frottement
tau_pred, u_tau_pred = tau_and_u_tau(cf_pred, 1, 1000)  # tau et u_tau estimés
print(f"Reynolds : Re_H = {Re_H:.1e}\n\nEstimations :\n\tcf = {cf_pred:.4f} \ttau_w = {tau_pred:.2f} Pa\t\tu_tau = {u_tau_pred:.3f} m/s")
print("\nMaillages :\n Valeurs estimées de y+ à la paroi :\n")
y_wall_list = {}  # hauteur maille à la paroi
y_plus_list = {}  # y+ paroi
y_wall_list[vef] = y_wall_list[vdf] = y_wall_list[plm] = [0 for x in range(len(Ny))]
y_plus_list[vef] = y_plus_list[vdf] = y_plus_list[plm] = [0 for x in range(len(Ny))]

for i in range(len(Ny)):
    y_VDF, y_VEF, y_PolyMAC = y_wall(y1, Ny[i])  # valeurs de y à l'endroit où la vitesse à la paroi est calculée
    y_wall_list[vef][i], y_wall_list[vdf][i], y_wall_list[plm][i] = (
        y_VEF,
        y_VDF,
        y_PolyMAC,
    )
    y_VDF_plus, y_VEF_plus, y_PolyMAC_plus = y_plus_prediction(y1, Ny[i], u_tau_pred, mu, rho)  # y+ correspondant à y_wall calculé
    y_plus_list[vef][i], y_plus_list[vdf][i], y_plus_list[plm][i] = (
        y_VEF_plus,
        y_VDF_plus,
        y_PolyMAC_plus,
    )
    print(f" - mesh_{i} : y+_VDF = {y_VDF_plus:.1f}\t y+_VEF = {y_VEF_plus:.1f}\t y+_PolyMAC = {y_PolyMAC_plus:.1f}")
    check_mesh_param(Ny[i], taux1[i], taux2[i])

## Initialisation et Lancement des calculs

In [ ]:
force_recalculation = True  # put True to force recalculation
build = run.BUILD_DIRECTORY  # build directory (absolute path !)

In [ ]:
import pandas as pd

params = [
    configs,
    Ny,
    taux1,
    taux2,
    inlet_velocity,
    outlet_pressure,
    inlet_k,
    inlet_epsilon,
    x_prof,
    mu,
    rho,
    y_min_prof,
    y_max_prof,
    nb_points_prof,
    tmax,
]
# Substitution dictionnary : nous avons 2 jdd de base qui seront modifiés pour lancer les différentes config
dict_list = substitution(params)

import os

if force_recalculation or not os.path.exists(build):
    run.reset()  # Delete the build directory and empty the list of cases to be executed
    run.initBuildDirectory()
    for i, (dis, model, pb) in enumerate(configs):
        for j in range(len(Ny)):
            jdd = f"canal_plan_pb_{pb}.data"
            tc = run.addCaseFromTemplate(
                jdd,
                f"{dis}_{model}_{pb}/mesh_{j}",
                dict_list[i][j],
                nbProcs=number_of_partitions,
            )
            if number_of_partitions > 1:
                tc.partition()
    run.printCases()
    run.runCases()  # lance les différents cas test écrits dans build/
    perf = run.tablePerf()  # tableau des performances de calcul
    perf.to_pickle(f"{run.BUILD_DIRECTORY}/table_perf.pkl")
else:
    print("Using previous calculation")
    perf = pd.read_pickle(f"{run.BUILD_DIRECTORY}/table_perf.pkl")

display(perf)


### Evolution des résidus

In [ ]:
a = plot.Graph()
for i, (dis, model, pb) in enumerate(configs):
    for j in range(len(Ny)):
        a.addResidu(
            f"{build}/{dis}_{model}_{pb}/mesh_{j}/" + ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}.dt_ev",
            label=f"{dis} {model} {pb} mesh_{j}",
            color=colors[i],
            marker=markers[j],
        )
    a.label("time", "maxResidu")
    a.scale(yscale="log")

### Maillage
décommenter pour voir le maillage

In [ ]:
# from trustutils import visit
# for i in range(nb_method_Pb_hydr):
#    visit.showMesh(build+f"/{config[i]}/mesh_0/{file_pb_hydr}.lata","dom")
# for i in range(nb_method_Pb_multi):
#    visit.showMesh(build+f"/{config[i+2]}/mesh_0/{file_pb_multi}.lata","dom")

## Post-traitement des résultats
### Contraintes pariétales
Vérifier que la couche limite est bien développée : tracé de tau_wall(x)

In [ ]:
# # Calcul de u_tau (noté u*) pour les différents maillages
# # tracé de tau_w le long de la paroi : vérifier que la couche limite est bien developpée

print(
    f"Calcul des valeurs de u_tau à x={x_prof}m ainsi que y+ à la fontière sup de la 1ère maille pour VDF et PolyMAC et à la hauteur des centres des faces pour VEF :\n\n"
)
u_star_list = {}
y_plus_cal_list = {}

# valeur min et max de tau (sert à mettre toutes les courbes à la meme échelle)
min_tau = 50
max_tau = 0

fig = plt.figure(figsize=(15, 4 * ceil(len(configs) / 2)))
axs = fig.subplots(ceil(len(configs) / 2), 2)
for i, (dis, model, pb) in enumerate(configs):
    for j in range(len(Ny)):
        jdd = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        if pb == pbt:  # avec pb_hydraulique_turbulent : tau_w déduit des données dans le fichier Ustar.face
            # plot tau_x=f(x)
            wall_file = f"{build}/{dis}_{model}_{pb}/mesh_{j}/{jdd}_pb_Ustar.face"
            data = read_facefile(wall_file)
            keys = list(data.keys())
            x, y, u_plus, d_plus, u_star, tau, tau_x, tau_y = get_wall_param(data[keys[-1]])
            tau_x = rho * tau_x
            # save value of u_tau
            u_plus, d_plus, u_star, tau, tau_xprof, tau_yprof = get_values_at_x(data[keys[-1]], x_prof)
            u_star_list[(dis, model, pb, j)] = u_star
            if dis == vef:
                y_plus = y_wall_list[vef][j] * u_star * rho / mu
            elif dis == vdf:
                y_plus = y_wall_list[vdf][j] * u_star * rho / mu
            else:
                y_plus = y_wall_list[plm][j] * u_star * rho / mu
            y_plus_cal_list[(dis, model, pb, j)] = y_plus
            print(f"{dis:10}{model:10}{pb:7}mesh_{j} : u* = {u_star:8.4f} m/s, y+={y_plus:6.1f}")
        else:  # avec pb_hmultiphase : tau_w déduit de la sonde de y+
            # plot tau_x=f(x)
            x, y_demi = abscisses_sonde_yplus((dis, model, pb), Ny[j])
            y_plus = np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{jdd}_Y_PLUS.son")[1::, 1])
            u_tau, tau_x = get_tau_from_yplus(y_plus, y_demi, rho, mu)
            # save value of u_tau
            u_star = get_value_at_x(u_tau, x, x_prof)
            u_star_list[(dis, model, pb, j)] = u_star
            y_plus = 2 * y_demi * u_star * rho / mu
            y_plus_cal_list[(dis, model, pb, j)] = y_plus
            print(
                f"{dis:10}{model:10}{pb:7}mesh_{j} : u* = {u_star:8.4f} m/s, y+={y_plus:6.1f}"
            )  # 2*y_demi pour etre à la frontière sup de la 1ere maille à la paroi
        if min(tau_x) < min_tau:
            min_tau = min(tau_x)
        if max(tau_x) > max_tau:
            max_tau = max(tau_x)
        axs[int(i / 2), i % 2].plot(x, tau_x, label=f"mesh_{j}")  # Change plotted variable here (choose u_plus, d_plus, u_star, tau, tau_x or tau_y)
    axs[int(i / 2), i % 2].plot(x, tau_pred * np.ones(len(x)), "--", label="estimation")
    axs[int(i / 2), i % 2].legend(fontsize=12)
    axs[int(i / 2), i % 2].set_xlabel("x [m]", fontsize=10)
    axs[int(i / 2), i % 2].set_ylabel("tau_x [Pa]", fontsize=10)
    axs[int(i / 2), i % 2].set_title(f"{dis} {model} {pb}", fontsize=12)

# Defining custom 'xlim' and 'ylim' values.
custom_xlim = (0, 100)
custom_ylim = (min_tau - 0.1, max_tau + 0.1)

# Setting the values for all axes.
plt.setp(axs, xlim=custom_xlim, ylim=custom_ylim)

fig.suptitle(f"Contraintes pariétales à t={round(keys[-1],1)}s")
fig.tight_layout()
print(f"\nComparaison avec la estimation : u* = {u_tau_pred:.4f} m/s")
for i in range(len(Ny)):
    print(
        f" - mesh_{i} y+ à la paroi : y+_VEF = {y_plus_list[vef][i]:6.1f}   y+_VDF = {y_plus_list[vdf][i]:6.1f}\t y+_PolyMAC = {y_plus_list[plm][i]:.1f}"
    )

### Profils de vitesse
#### Etude de la convergence en maillage

In [ ]:
# # Plot u⁺=f(y⁺)
fig = plt.figure(figsize=(15, 4 * ceil(len(configs) / 2)))
axs = fig.subplots(ceil(len(configs) / 2), 2)

for i, (dis, model, pb) in enumerate(configs):
    for j in range(len(Ny)):
        file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        utau = u_star_list[(dis, model, pb, j)]
        uplus = np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_VITESSE.son")[1::2, 1]) / utau
        yplus = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j]) * utau / (mu / rho)
        axs[int(i / 2), i % 2].plot(yplus, uplus, "-o", markersize=3, label=f"mesh_{j}: u⁺=f(y⁺)")
        axs[int(i / 2), i % 2].plot(
            y_plus_cal_list[(dis, model, pb, j)] * np.ones(2),
            np.linspace(4, 20, 2),
            linestyle="dotted",
            label=f"mesh_{j}: y⁺wall={y_plus_cal_list[(dis, model, pb, j)]:.0f}",
        )
    axs[int(i / 2), i % 2].plot(
        yplus[yplus > 10],
        (1 / kappa) * np.log(yplus[yplus > 10]) + B,
        "--",
        label="log law",
        color="black",
    )
    # axs[int(i/2),i%2].plot(yplus[yplus<15], yplus[yplus<15], "--", label="u⁺ = y⁺", color="grey")
    axs[int(i / 2), i % 2].legend(fontsize=10)
    axs[int(i / 2), i % 2].set_xlabel("y+", fontsize=12)
    axs[int(i / 2), i % 2].set_ylabel("u+", fontsize=12)
    axs[int(i / 2), i % 2].set_xscale("log")
    axs[int(i / 2), i % 2].set_title(f"{dis} {model} {pb}", fontsize=12)
fig.suptitle(f"Profils de vitesse à x={x_prof}m : convergence en maillage")
fig.tight_layout()

In [ ]:
# # Plot u=f(y)

Graph = plot.Graph(
    "Velocity profiles : convergence en maillage",
    nX=ceil(len(configs) / 2),
    nY=2,
    size=[8, 6],
)
for i, (dis, model, pb) in enumerate(configs):
    file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
    Graph.addPlot((int(i / 2), i % 2), f"{dis} {model} {pb}")
    for j in range(len(Ny)):
        Graph.addSegment(
            f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_VITESSE.son",
            linestyle="solid",
            marker="o",
            markersize=3,
            compo=0,
            label=f"mesh_{j}",
        )
    Graph.legend()
    Graph.label("y [m]", "u [m/s]")

#### Comparaison des différentes méthodes de calcul

In [ ]:
# # Plot u+=f(y+)

Graph = plot.Graph("Velocity profiles : comparaison des méthodes", nY=len(Ny), size=[8, 5])

for j in range(len(Ny)):
    Graph.addPlot(j, f"mesh_{j}")
    for i, (dis, model, pb) in enumerate(configs):
        jdd = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        utau = u_tau_pred
        # if pb == pbt:
        utau = u_star_list[(dis, model, pb, j)]
        uplus = np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{jdd}_VITESSE.son")[1::2, 1]) / utau
        yplus = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j]) * utau / (mu / rho)
        Graph.add(
            yplus,
            uplus,
            linestyle="solid",
            marker="o",
            markersize=3,
            label=f"{dis} {model} {pb}",
        )
    Graph.add(
        yplus[yplus > 10],
        (1 / kappa) * np.log(yplus[yplus > 10]) + B,
        "--",
        linewidth=2,
        label="log law",
        color="black",
    )
    Graph.scale(xscale="log")
    Graph.legend()
    Graph.label("y⁺", "u⁺")

In [ ]:
# # Plot u=f(y)

Graph = plot.Graph("Velocity profiles : comparaison des méthodes", nY=len(Ny), size=[8, 6])
for j in range(len(Ny)):
    Graph.addPlot(j, f"mesh_{j}")
    for i, (dis, model, pb) in enumerate(configs):
        jdd = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        Graph.addSegment(
            f"{build}/{dis}_{model}_{pb}/mesh_{j}/{jdd}_VITESSE.son",
            compo=0,
            linestyle="solid",
            marker="o",
            markersize=3,
            label=f"{dis} {model} {pb}",
        )
    Graph.legend()
    Graph.label("y [m]", "u [m/s]")

### Traceurs de la turbulence
#### Etude de la convergence en maillage

In [ ]:
Graph = plot.Graph(
    title="Traceurs de turbulence : convergence en maillage",
    nY=2,
    nX=len(configs),
)

for i, (dis, model, pb) in enumerate(configs):
    Graph.addPlot((i, 0), f"{dis} {model} {pb}")
    for j in range(len(Ny)):
        file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        Graph.addSegment(
            f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_K.son",
            linestyle="solid",
            marker="o",
            markersize=3,
            label=f"mesh_{j}",
        )
    Graph.label("y [m]", "k [m²/s²]")


for i, (dis, model, pb) in enumerate(configs):
    Graph.addPlot((i, 1), f"{dis} {model} {pb}")
    for j in range(len(Ny)):
        file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        diss = np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_DISS.son")[1::, 1])
        k = np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_K.son")[1::, 1])
        if "tau" in model:
            epsilon = 0.09 * k / diss
        elif "omega" in model:
            epsilon = 0.09 * k * diss
        else:
            epsilon = diss
        y = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])
        Graph.add(y, epsilon, "-o", markersize=3, label=f"mesh_{j}")
    Graph.label("y [m]", "epsilon [m²/s³]")

#### Comparaison des différentes méthodes de calcul

In [ ]:
Graph = plot.Graph(
    "Traceurs de la turbulence : comparaison des méthodes",
    nX=2,
    nY=len(Ny),
    size=[8, 6],
)
for j in range(len(Ny)):
    if len(Ny) == 1:
        indice = 0
    else:
        indice = (0, j)
    Graph.addPlot(indice, f"mesh_{j}")
    for i, (dis, model, pb) in enumerate(configs):
        file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        Graph.addSegment(
            f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_K.son",
            linestyle="solid",
            marker="o",
            markersize=3,
            label=f"{dis} {model} {pb}",
        )
    Graph.legend()
    Graph.label("y [m]", "k [m²/s²]")

for j in range(len(Ny)):
    if len(Ny) == 1:
        indice = 1
    else:
        indice = (1, j)
    Graph.addPlot(indice, f"mesh_{j}")
    for i, (dis, model, pb) in enumerate(configs):
        file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        diss = np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_DISS.son")[1::, 1])
        k = np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_K.son")[1::, 1])
        if "tau" in model:
            epsilon = 0.09 * k / diss
        elif "omega" in model:
            epsilon = 0.09 * k * diss
        else:
            epsilon = diss
        y = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])
        Graph.add(y, epsilon, "-o", markersize=3, label=f"{dis} {model} {pb}")
    Graph.legend()
    Graph.label("y [m]", "epsilon [m²/s³]")

### Perte de charge
Les pertes de charge sont calculées entre la sortie et une section de l'écoulement à 5m de l'entrée car VEF_k-epsilon et VDF_k-tau prédisent mal le champ de pression en entrée

In [ ]:
Re_Dh = 4 * Re_H
Dh = 4 * H
f_guess = 0.02
f = facteur_frottement(Re_Dh, f_guess)  # coefficient de frottement donnée par la formule de Von Karman et Nikurade (Re>1e5)
v_m = inlet_velocity  # vitesse moyenne sur une section
L = 100 - 5  # [m] : longueur du conduit
delta_p_estime = -f * L / Dh * rho * v_m**2 / 2
print(f"Estimation de la perte de charge avec la formule de Von Karman et Nikuradse (comme le diagramme de Moody):\nDeltaP = {delta_p_estime:.1f} Pa")

In [ ]:
for j in range(len(Ny)):
    for i, (dis, model, pb) in enumerate(configs):
        file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
        p_inlet = np.mean(np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_PRESSION_I.son")[1::, 1]))
        p_outlet = np.mean(np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_PRESSION_O.son")[1::, 1]))
        delta_p = p_outlet - p_inlet
        if model == "k-epsilon":
            delta_p = rho * delta_p
        # delta_u2 = u[1]**2-u[0]**2
        print(
            f"{dis:10}{model:10}{pb:7} - mesh_{j} : deltaP = {delta_p:.1f} Pa Ecart relatif : {100*(abs(delta_p)-abs(delta_p_estime))/abs(delta_p_estime):.0f}%"
        )

### Pression
On verifie que le gradient de pression reste négligeable : dans ce cas la loi logarithmique doit bien être retrouvée.
Les valeurs maximales de p+ (paramètre de Mellor & Gibson) sont calculées.

In [ ]:
# # Plot P(x)

Graph = plot.Graph("Pressure evolution at y=0.8m", subtitle="pressure", nX=len(configs))
for i, (dis, model, pb) in enumerate(configs):
    file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
    Graph.addPlot(i, f"{dis} {model} {pb}")
    for j in range(len(Ny)):
        Graph.addSegment(
            f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_PRESSION.son",
            label=f"mesh_{j}",
        )
    Graph.legend()
    Graph.label("x [m]", "P/rho [Pa.m³/kg]")

# Graph = plot.Graph("Pressure gradient at y=0.8m", subtitle="pressure", nX=len(configs))
# for i, (dis, model, pb) in enumerate(configs):
#     if "VEF" in dis:
#         file = ("PAR_" if number_of_partitions > 1 else "") + f"canal_plan_pb_{pb}"
#         Graph.addPlot(i, f"{dis} {model} {pb}")
#         for j in range(len(Ny)):
#             Graph.addSegment(
#                 f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_GRADP.son",
#                 label=f"mesh_{j}",
#             )
#             Graph.legend()
#             Graph.label("x [m]", "grad(P/rho) [m/s²]")
#             dp_dx = np.array(plot.loadText(f"{build}/{dis}_{model}_{pb}/mesh_{j}/{file}_GRADP.son")[1::, 1])
#             p_plus = (mu / rho) * np.max(np.abs(dp_dx)) / u_star_list[i][j]
#             print(f"{dis} {model} {pb}, mesh_{j} : p+={p_plus:.2e}")

In [ ]:
# fig=visit.Show(build+f"/VDF_k-epsilon/mesh_0/{file_pb_hydr}.lata","Pseudocolor","VITESSE_X_ELEM_dom")

# to add a list of viewing options
# fig.visuOptions(['no_databaseinfo','no_legend'])

# To display the visu with the initialization visit.Show()
# fig.plot()